In [1]:
import numpy as np, pandas as pd, torch
from torch.distributions import Categorical


df = pd.read_csv("avg_pay_by_hour_of_week.csv")

df = df.sort_values("hour_of_week").reset_index(drop=True)

N = 168                          # hours in week
L_hours = 10                     # shift length in hours
K_shifts = 6                     # number of shifts to pick
L = L_hours                      # because prices are per-hour rows
pay = df["pay_per_minute"].to_numpy().astype(float)  # optimize pay/min
labels = df["hour_label"].tolist()


window_val = np.array([pay.take([(i+j) % N]) for i in range(N) for j in range(L)]).reshape(N, L).sum(axis=1)


def dp_line(start, length, max_take):
    start_positions = np.arange(start, start + max(0, length - L + 1))
    M = len(start_positions)
    if M == 0 or max_take == 0:
        return 0.0, []


    next_idx = np.searchsorted(start_positions, start_positions + L, side="left")
    dp = np.zeros((M + 1, max_take + 1))
    take = np.zeros((M + 1, max_take + 1), dtype=bool)

    for i in range(M - 1, -1, -1):
        wi = window_val[start_positions[i] % N]
        for k in range(1, max_take + 1):
            skip = dp[i + 1, k]
            take_val = wi + dp[next_idx[i], k - 1]
            if take_val > skip:
                dp[i, k] = take_val
                take[i, k] = True
            else:
                dp[i, k] = skip

    res = []
    i, k = 0, max_take
    while i < M and k > 0:
        if take[i, k]:
            res.append(start_positions[i] % N)
            i = next_idx[i]
            k -= 1
        else:
            i += 1
    return float(dp[0, max_take]), res

def solve_exact():
    bestA, startsA = dp_line(0, N, K_shifts)
    bestB, bestB_starts = -1.0, None
    for s in range(N - L + 1, N):  
        val_rest, rest_starts = dp_line(s + L, N - L, K_shifts - 1)
        total = window_val[s] + val_rest
        if total > bestB:
            bestB = total
            bestB_starts = [s % N] + rest_starts

    if bestB > bestA:
        total_val, starts = bestB, sorted(bestB_starts)
    else:
        total_val, starts = bestA, sorted(startsA)

    def shift_label(s):
        return f"{labels[s]} to {labels[(s + L - 1) % N]}"

    per_shift_pay_usd = [60.0 * window_val[s] for s in starts] 
    total_usd = float(np.sum(per_shift_pay_usd))
    return starts, per_shift_pay_usd, total_usd, [shift_label(s) for s in starts]

starts, per_shift_pay_usd, weekly_total_usd, nice_labels = solve_exact()
print("EXACT optimal 6x10h shifts:")
for s, amt, txt in zip(starts, per_shift_pay_usd, nice_labels):
    print(f"  start={s:3d}  ~${amt:,.2f}   {txt}")
print(f"\nWeekly total (pay only): ${weekly_total_usd:,.2f}")


EXACT optimal 6x10h shifts:
  start= 20  ~$536.10   Monday 20:00-21:00 to Tuesday 05:00-06:00
  start= 45  ~$541.30   Tuesday 21:00-22:00 to Wednesday 06:00-07:00
  start= 93  ~$540.85   Thursday 21:00-22:00 to Friday 06:00-07:00
  start=118  ~$541.97   Friday 22:00-23:00 to Saturday 07:00-08:00
  start=142  ~$567.57   Saturday 22:00-23:00 to Sunday 07:00-08:00
  start=164  ~$556.68   Sunday 20:00-21:00 to Monday 05:00-06:00

Weekly total (pay only): $3,284.47


Finding Optimal "Day Time Shifts" (after 8AM start before 2AM end)

In [2]:
import numpy as np, pandas as pd, torch
from torch.distributions import Categorical
from io import StringIO


L_hours = 10          
K_shifts = 6          
LATEST_NIGHT_HOUR = 2 # allow 00:00–02:00 but NOT past 02:00
EARLIEST_DAY_HOUR = 8 # allow 06:00–24:00

df = pd.read_csv("avg_pay_by_hour_of_week.csv")
df = df.sort_values("hour_of_week").reset_index(drop=True)


N = 168
L = L_hours
pay = df["pay_per_minute"].to_numpy(float)
labels = df["hour_label"].tolist()


window_val = np.array([pay.take([(i+j) % N]) for i in range(N) for j in range(L)]).reshape(N, L).sum(axis=1)


def build_allowed_mask(latest_night_hour, earliest_day_hour):
    mask = np.ones(24, dtype=bool)
    if latest_night_hour is not None or earliest_day_hour is not None:
        mask[:] = False
        if earliest_day_hour is not None:
            mask[earliest_day_hour:24] = True
        if latest_night_hour is not None:
            mask[0:max(0, int(latest_night_hour))] = True
    return mask

allowed_hour_mask = build_allowed_mask(LATEST_NIGHT_HOUR, EARLIEST_DAY_HOUR)

SHIFT_OK = np.array([
    all(allowed_hour_mask[(s + j) % 24] for j in range(L))
    for s in range(N)
], dtype=bool)

def dp_line(start, length, max_take):
    all_pos = np.arange(start, start + max(0, length - L + 1))
    starts = np.array([p for p in all_pos if SHIFT_OK[p % N]], dtype=int)
    M = len(starts)
    if M == 0 or max_take == 0:
        return 0.0, []

    next_idx = np.searchsorted(starts, starts + L, side="left")
    dp = np.zeros((M + 1, max_take + 1))
    take = np.zeros((M + 1, max_take + 1), dtype=bool)

    for i in range(M - 1, -1, -1):
        wi = window_val[starts[i] % N]
        for k in range(1, max_take + 1):
            skip = dp[i + 1, k]
            take_val = wi + dp[next_idx[i], k - 1]
            if take_val > skip:
                dp[i, k] = take_val; take[i, k] = True
            else:
                dp[i, k] = skip

    res, i, k = [], 0, max_take
    while i < M and k > 0:
        if take[i, k]:
            res.append(starts[i] % N)
            i = next_idx[i]; k -= 1
        else:
            i += 1
    return float(dp[0, max_take]), res

def solve_exact():
    bestA, startsA = dp_line(0, N, K_shifts)
    bestB, bestB_starts = -1.0, None
    for s in range(N - L + 1, N):
        if not SHIFT_OK[s % N]: 
            continue
        val_rest, rest_starts = dp_line(s + L, N - L, K_shifts - 1)
        total = window_val[s] + val_rest
        if total > bestB:
            bestB, bestB_starts = total, [s % N] + rest_starts

    if bestB > bestA and bestB_starts is not None:
        total_val, starts = bestB, sorted(bestB_starts)
    else:
        total_val, starts = bestA, sorted(startsA)

    def shift_label(s):
        return f"{labels[s]} to {labels[(s + L - 1) % N]}"

    per_shift_pay_usd = [60.0 * window_val[s] for s in starts]
    return starts, per_shift_pay_usd, float(np.sum(per_shift_pay_usd)), [shift_label(s) for s in starts]

starts, per_shift_pay_usd, weekly_total_usd, nice_labels = solve_exact()
print(f"EXACT optimal 6x{L}h (allowed hours: "
      f"{'all' if (LATEST_NIGHT_HOUR is None and EARLIEST_DAY_HOUR is None) else f'[0,{LATEST_NIGHT_HOUR}) ∪ [{EARLIEST_DAY_HOUR},24)'}):")
for s, amt, txt in zip(starts, per_shift_pay_usd, nice_labels):
    print(f"  start={s:3d}  ~${amt:,.2f}   {txt}")
print(f"\nWeekly total: ${weekly_total_usd:,.2f}")


EXACT optimal 6x10h (allowed hours: [0,2) ∪ [8,24)):
  start= 16  ~$469.55   Monday 16:00-17:00 to Tuesday 01:00-02:00
  start= 40  ~$459.26   Tuesday 16:00-17:00 to Wednesday 01:00-02:00
  start= 88  ~$467.93   Thursday 16:00-17:00 to Friday 01:00-02:00
  start=112  ~$478.99   Friday 16:00-17:00 to Saturday 01:00-02:00
  start=136  ~$503.28   Saturday 16:00-17:00 to Sunday 01:00-02:00
  start=160  ~$495.47   Sunday 16:00-17:00 to Monday 01:00-02:00

Weekly total: $2,874.48


Finding Optimal "Day Time Shifts" (after 8AM start before 10PM end)

In [3]:
import numpy as np, pandas as pd, torch
from torch.distributions import Categorical
from io import StringIO


L_hours = 10         
K_shifts = 6          
EARLIEST_START_HOUR = 8
LATEST_END_HOUR     = 22   


h = np.arange(N) % 24                   
SHIFT_OK = (h >= EARLIEST_START_HOUR) & ((h + L_hours) <= LATEST_END_HOUR)

allowed_starts_hod = np.unique(h[SHIFT_OK])
print("Allowed start hours-of-day:", allowed_starts_hod.tolist())


df = pd.read_csv("avg_pay_by_hour_of_week.csv")
df = df.sort_values("hour_of_week").reset_index(drop=True)


N = 168
L = L_hours
pay = df["pay_per_minute"].to_numpy(float)
labels = df["hour_label"].tolist()

window_val = np.array([pay.take([(i+j) % N]) for i in range(N) for j in range(L)]).reshape(N, L).sum(axis=1)


def dp_line(start, length, max_take):
    all_pos = np.arange(start, start + max(0, length - L + 1))
    starts = np.array([p for p in all_pos if SHIFT_OK[p % N]], dtype=int)
    M = len(starts)
    if M == 0 or max_take == 0:
        return 0.0, []

    next_idx = np.searchsorted(starts, starts + L, side="left")
    dp = np.zeros((M + 1, max_take + 1))
    take = np.zeros((M + 1, max_take + 1), dtype=bool)

    for i in range(M - 1, -1, -1):
        wi = window_val[starts[i] % N]
        for k in range(1, max_take + 1):
            skip = dp[i + 1, k]
            take_val = wi + dp[next_idx[i], k - 1]
            if take_val > skip:
                dp[i, k] = take_val; take[i, k] = True
            else:
                dp[i, k] = skip

    res, i, k = [], 0, max_take
    while i < M and k > 0:
        if take[i, k]:
            res.append(starts[i] % N)
            i = next_idx[i]; k -= 1
        else:
            i += 1
    return float(dp[0, max_take]), res

def solve_exact():
    bestA, startsA = dp_line(0, N, K_shifts)
    bestB, bestB_starts = -1.0, None
    for s in range(N - L + 1, N):
        if not SHIFT_OK[s % N]: 
            continue
        val_rest, rest_starts = dp_line(s + L, N - L, K_shifts - 1)
        total = window_val[s] + val_rest
        if total > bestB:
            bestB, bestB_starts = total, [s % N] + rest_starts

    if bestB > bestA and bestB_starts is not None:
        total_val, starts = bestB, sorted(bestB_starts)
    else:
        total_val, starts = bestA, sorted(startsA)

    def shift_label(s):
        return f"{labels[s]} to {labels[(s + L - 1) % N]}"

    per_shift_pay_usd = [60.0 * window_val[s] for s in starts]
    return starts, per_shift_pay_usd, float(np.sum(per_shift_pay_usd)), [shift_label(s) for s in starts]

starts, per_shift_pay_usd, weekly_total_usd, nice_labels = solve_exact()
print(f"EXACT optimal 6x{L}h (allowed hours: "
      f"{'all' if (LATEST_NIGHT_HOUR is None and EARLIEST_DAY_HOUR is None) else f'[0,{LATEST_NIGHT_HOUR}) ∪ [{EARLIEST_DAY_HOUR},24)'}):")
for s, amt, txt in zip(starts, per_shift_pay_usd, nice_labels):
    print(f"  start={s:3d}  ~${amt:,.2f}   {txt}")
print(f"\nWeekly total: ${weekly_total_usd:,.2f}")


Allowed start hours-of-day: [8, 9, 10, 11, 12]
EXACT optimal 6x10h (allowed hours: [0,2) ∪ [8,24)):
  start= 12  ~$414.95   Monday 12:00-13:00 to Monday 21:00-22:00
  start= 36  ~$401.99   Tuesday 12:00-13:00 to Tuesday 21:00-22:00
  start= 84  ~$405.72   Thursday 12:00-13:00 to Thursday 21:00-22:00
  start=108  ~$426.83   Friday 12:00-13:00 to Friday 21:00-22:00
  start=132  ~$440.30   Saturday 12:00-13:00 to Saturday 21:00-22:00
  start=156  ~$452.63   Sunday 12:00-13:00 to Sunday 21:00-22:00

Weekly total: $2,542.41
